# FORMULACIÓN DEL MODELO DE OPTIMIZACIÓN

La formulación del modelo se basa en la representación matemática de las relaciones entre actores de la red planteada. Para esto se formula un algoritmos de programación lineal de enteros mixtos (MILP) que contemple el problema de asignación de cantidad optima de cada tipo de queso y operación de agentes dentro de la red, para llegar a la modelación definitiva se lleva a cabo las distintas pruebas de funcionalidad con el fin de validad cada una de las restricciones, supuestos y variables de salida, con el fin de verificar la validez de los resultados. 

## Modelo de optimización

### 1. Importa la librería 

In [21]:
# Instalando biblioteca pyomo y GLPK
# Instalando Pyomo
# pip3 install pyomo
# pip3 install pandas

In [22]:

import pandas as pd
from pyomo.environ import*
import pyomo.environ as pyo

### 2. Crea un modelo de Pyomo 

In [23]:
# Crear un modelo abstracto con Pyomo
modelo = pyo.AbstractModel()

### 3. Cargamos los parametros

In [24]:
# Cargar datos desde archivos CSV
# Asumiendo que tienes archivos CSV llamados 'costo_produccion.csv', 'costo_fijo_acopio.csv', etc.
# Ajusta los nombres de archivos según tus necesidades

productores_df = pd.read_csv(r'C:\Users\UNIMAG\Desktop\HENRY\OptimizationModel\Optimization models (Pyomo)\CORDOBA\Productores.csv', index_col='Productores')
centros_de_acopios_df = pd.read_csv(r'C:\Users\UNIMAG\Desktop\HENRY\OptimizationModel\Optimization models (Pyomo)\CORDOBA\CAcopios.csv', index_col='CAcopios')
clientes_df = pd.read_csv(r'C:\Users\UNIMAG\Desktop\HENRY\OptimizationModel\Optimization models (Pyomo)\CORDOBA\Clientes.csv')
productos_df = pd.read_csv(r'C:\Users\UNIMAG\Desktop\HENRY\OptimizationModel\Optimization models (Pyomo)\CORDOBA\Productos.csv')
periodo_de_tiempo_df = pd.read_csv(r'C:\Users\UNIMAG\Desktop\HENRY\OptimizationModel\Optimization models (Pyomo)\CORDOBA\Tiempo.csv')

costo_produccion_df = pd.read_csv(r'C:\Users\UNIMAG\Desktop\HENRY\OptimizationModel\Optimization models (Pyomo)\CORDOBA\CostoProduccion.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
costo_fijo_acopio_df = pd.read_csv(r'C:\Users\UNIMAG\Desktop\HENRY\OptimizationModel\Optimization models (Pyomo)\CORDOBA\CostoFijoAcopio.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
costo_var_acopio_df = pd.read_csv(r'C:\Users\UNIMAG\Desktop\HENRY\OptimizationModel\Optimization models (Pyomo)\CORDOBA\CostoVarAcopio.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
costo_inv_acopio_df = pd.read_csv(r'C:\Users\UNIMAG\Desktop\HENRY\OptimizationModel\Optimization models (Pyomo)\CORDOBA\CostoInvAcopio.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
costo_trans_prod_acopio_df = pd.read_csv(r'C:\Users\UNIMAG\Desktop\HENRY\OptimizationModel\Optimization models (Pyomo)\CORDOBA\CostoTransProdAcopio.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
costo_trans_acopio_clie_df = pd.read_csv(r'C:\Users\UNIMAG\Desktop\HENRY\OptimizationModel\Optimization models (Pyomo)\CORDOBA\CostoTransAcopioClie.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
demanda_clie_df = pd.read_csv(r'C:\Users\UNIMAG\Desktop\HENRY\OptimizationModel\Optimization models (Pyomo)\CORDOBA\DemandaClie.csv', delimiter=';', decimal=',', encoding='utf-8-sig') 
cap_productor_df = pd.read_csv(r'C:\Users\UNIMAG\Desktop\HENRY\OptimizationModel\Optimization models (Pyomo)\CORDOBA\CapProductor.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
cap_almacenamientoCA_df = pd.read_csv(r'C:\Users\UNIMAG\Desktop\HENRY\OptimizationModel\Optimization models (Pyomo)\CORDOBA\CapAlmacenamientoCA.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
productos_dinsatisfecha_df = pd.read_csv(r'C:\Users\UNIMAG\Desktop\HENRY\OptimizationModel\Optimization models (Pyomo)\CORDOBA\PDinsatisfecha.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  

# Convertir los DataFrames a diccionarios

costo_produccion_dict = costo_produccion_df.set_index(['Productos', 'Productores', 'Tiempo'])['CostoProduccion'].to_dict()
costo_fijo_acopio_dict = costo_fijo_acopio_df.set_index(['CAcopios', 'Tiempo'])['CostoFijoAcopio'].to_dict()
costo_var_acopio_dict = costo_var_acopio_df.set_index(['Productos', 'CAcopios', 'Tiempo'])['CostoVarAcopio'].to_dict()
costo_inv_acopio_dict = costo_inv_acopio_df.set_index(['Productos', 'CAcopios', 'Tiempo'])['CostoInvAcopio'].to_dict()
costo_trans_prod_acopio_dict = costo_trans_prod_acopio_df.set_index(['Productores', 'Productos', 'CAcopios', 'Tiempo'])['CostoTransProdAcopio'].to_dict()
costo_trans_acopio_clie_dict = costo_trans_acopio_clie_df.set_index(['Productos', 'Productores', 'Clientes', 'Tiempo'])['CostoTransAcopioClie'].to_dict()
demanda_clie_dict = demanda_clie_df.set_index(['Productos', 'Clientes', 'Tiempo'])['DemandaClie'].to_dict()
cap_productor_dict = cap_productor_df.set_index(['Productores', 'Productos', 'Tiempo'])['CapProductor'].to_dict()
cap_almacenamientoCA_dict = cap_almacenamientoCA_df.set_index(['Productos', 'CAcopios', 'Tiempo'])['CapAlmacenamientoCA'].to_dict()
productos_dinsatisfecha_dict = productos_dinsatisfecha_df.set_index(['Productos', 'Tiempo'])['PDinsatisfecha'].to_dict()


### 4. Conjuntos

| Término | Descripción                    |
|---------|--------------------------------|
| I       | Conjunto de productores        |
| J       | Conjunto de centros de acopio  |
| K       | Conjunto de clientes           |
| P       | Conjunto de productos          |
| T       | Conjunto de periodos de tiempo |


In [25]:

modelo.productores = pyo.Set()  # Conjunto I de productores
modelo.centros_de_acopios = pyo.Set()  # Conjunto J de centros de acopio
modelo.clientes = pyo.Set()  # Conjunto K de clientes
modelo.productos = pyo.Set()  # Conjunto P de productos
modelo.periodo_de_tiempo = pyo.Set()  # Conjunto T de periodos de tiempo

### 5. Parámetros 

| Término                       | Descripción                                                                       | Unidades    |
|-------------------------------|-----------------------------------------------------------------------------------|-------------|
| Costos de producción          |                                                                               |             |
| $CostoProducción_{pit}$        | Costo de producción del producto p en el productor i en el periodo de tiempo t    | [$/kg]      |
| Costos de operación           |                                                                               |             |
| $CostoFijoAcopi_{jt}$         | Costo fijo del centro de acopio j en el periodo de tiempo t                        | [$]         |
| $CostoVarAcopio_{pjt}$         | Costo variable del producto p en el centro de acopio j en el periodo de tiempo t  | [$/kg]      |
| Costos de inventario          |                                                                               |             |
| $CostoInvAcopio_{pjt}$         | Costo de inventario del producto p en el centro de acopio j en el periodo de tiempo t | [$/kg]   |
| Costos de transporte          |                                                                               |             |
| $CostoTransProdAcopi_{pijt}$  | Costo de transporte del producto p desde el productor i hasta el centro de acopio j en el periodo de tiempo t | [$/kg] |
| $CostoTransAcopioClie_{pjkt}$  | Costo de transporte del producto p desde el centro de acopio j hasta el cliente k en el periodo de tiempo t | [$/kg] |
| Demanda                       |                                                                               |             |
| $DemandaClie_{pkt}$            | Demanda del producto p en el cliente k en el periodo de tiempo t                  | [kg]        |
| Capacidades                   |                                                                               |             |
| $CapProductor_{pit}$           | Capacidad de suministro del producto p en el productor i en el periodo de tiempo t | [kg]        |
| $CapAlmacenamientoCA_{pjt}$    | Capacidad de almacenamiento del producto p en el centro de acopio j en el periodo de tiempo t | [kg]   |
| $PDinsatisfecha_{pt}$          | Costo de la demanda insatisfecha del producto p en el periodo de tiempo t         | [$/kg]      |


In [26]:

modelo.CostoProduccion = pyo.Param(modelo.productores, modelo.productos, modelo.periodo_de_tiempo, initialize = costo_produccion_dict)
modelo.CostoFijoAcopio = pyo.Param(modelo.centros_de_acopios, modelo.periodo_de_tiempo, initialize = costo_fijo_acopio_dict)
modelo.CostoVarAcopio = pyo.Param(modelo.productos, modelo.centros_de_acopios, modelo.periodo_de_tiempo, initialize = costo_var_acopio_dict)
modelo.CostoInvAcopio = pyo.Param(modelo.productos, modelo.centros_de_acopios, modelo.periodo_de_tiempo, initialize = costo_trans_prod_acopio_dict)
modelo.CostoTransProdAcopio = pyo.Param(modelo.productores, modelo.productos, modelo.centros_de_acopios, modelo.periodo_de_tiempo, initialize = costo_trans_acopio_clie_dict)
modelo.CostoTransAcopioClie = pyo.Param(modelo.centros_de_acopios, modelo.clientes, modelo.productos, modelo.periodo_de_tiempo, initialize = costo_trans_acopio_clie_dict)
modelo.DemandaClie = pyo.Param(modelo.productos, modelo.clientes, modelo.periodo_de_tiempo, initialize = demanda_clie_dict)
modelo.CapProductor = pyo.Param(modelo.productores, modelo.productos, modelo.periodo_de_tiempo, initialize = cap_productor_dict)
modelo.CapAlmacenamientoCA = pyo.Param(modelo.centros_de_acopios, modelo.productos, modelo.periodo_de_tiempo, initialize = cap_almacenamientoCA_dict)
modelo.PDinsatisfecha = pyo.Param(modelo.productos, modelo.periodo_de_tiempo, initialize = productos_dinsatisfecha_dict)

### 6. Variables de decisión 

| Término                 | Descripción                                                                       | Unidades  |
|-------------------------|-----------------------------------------------------------------------------------|-----------|
| Variables de operación  |                                                                               |           |
| $CA_{jt}$               | 1 si se opera un centro de acopio j en el periodo de tiempo t; 0 en caso contrario | [binaria] |
| $AS_{ijt}$              | 1 si se asigna un envío de i a j en el periodo de tiempo t; 0 en caso contrario   | [binaria] |
| Variables de flujo      |                                                                               |           |
| $PA_{pijt}$             | Cantidad del producto p por enviar desde el productor i hasta el centro de acopio j en el periodo de tiempo t | [kg]      |
| $AC_{pjkt}$             | Cantidad del producto p por enviar desde el centro de acopio j al cliente k en el periodo de tiempo t       | [kg]      |
| Variables de inventario |                                                                               |           |
| $InvCA_{pjt}$           | Inventario del producto p en el centro de acopio j en el periodo de tiempo t     | [kg]      |
| Variables demanda insatisfecha |                                                      |           |
| $DInsatisfecha_{pt}$    | Demanda insatisfecha del producto p en el periodo de tiempo t                    | [kg]      |



In [27]:

modelo.CA = pyo.Var(modelo.centros_de_acopios, modelo.periodo_de_tiempo, within=pyo.Binary)
modelo.AS = pyo.Var(modelo.productores, modelo.centros_de_acopios, modelo.periodo_de_tiempo, within=pyo.Binary)
modelo.PA = pyo.Var(modelo.productos, modelo.productores, modelo.centros_de_acopios, modelo.periodo_de_tiempo, within=pyo.NonNegativeReals)
modelo.AC = pyo.Var(modelo.productos, modelo.centros_de_acopios, modelo.clientes, modelo.periodo_de_tiempo, within=pyo.NonNegativeReals)
modelo.InvCA = pyo.Var(modelo.productos, modelo.centros_de_acopios, modelo.periodo_de_tiempo, within=pyo.NonNegativeReals)
modelo.DInsatisfecha = pyo.Var(modelo.productos, modelo.periodo_de_tiempo, within=pyo.NonNegativeReals)


### 7. Restricciones de capacidad y demanda

(2) De Capacidad de productores:
\begin{align*}
& PA_{pijt} = \text{CapProductor}_{pit} \cdot \text{AS}_{ijt} \quad \forall p \in P, \forall i \in I, \forall j \in J, \forall t \in T
\end{align*}

(3) De Flujo:

En los centros de acopio

\begin{align*}
& \sum_{i \in I} PA_{pijt} = \text{InvCA}_{pjt} + \sum_{k \in K} AC_{pjkt} \quad \forall p \in P, \forall j \in J, t=1 \
\\
\\
& \sum_{i \in I} PA_{pijt} + \text{InvCA}_{pjt-1} = \text{InvCA}_{pjt} + \sum_{k \in K} AC_{pjkt} \quad \forall p \in P, \forall j \in J, t \geq 2
\end{align*}

(4) De relación de variables binarias y de Operación:
\begin{align*}
& \sum_{i \in I} PA_{pijt} \leq CA_{jt} \cdot \text{CapAlmacenamientoCA}_{pjt} \quad \forall j \in J, \forall i \in I, \forall p \in P, \forall t \in T
\end{align*}

(5) Asignación:
\begin{align*}
& \sum_{j \in J} AS_{ijt} = 1 \quad \forall j \in J, \forall i \in I, \forall p \in P
\end{align*}

(6) Demanda insatisfecha:
\begin{align*}
& \sum_{j \in J} \sum_{k \in K} AC_{pjkt} + \text{Dinsatisfecha}_{pt} = \sum_{k \in K} \text{DemandaClie}_{pkt} \quad \forall t \in T, \forall p \in P
\end{align*}


In [28]:
def cap_productor_rule(modelo, p, i, j, t):
    return modelo.PA[p, i, j, t] <= modelo.CapProductor[i, p, t] * modelo.AS[i, j, t]

modelo.cap_productor_constraint = pyo.Constraint(modelo.productores, modelo.productos, modelo.periodo_de_tiempo, rule=cap_productor_rule)

def flujo_acopio_rule(modelo, p, j, t):
    if t == 1:
        return sum(modelo.PA[p, i, j, t] for i in modelo.productores) == modelo.InvCA[p, j, t] + sum(modelo.AC[p, j, k, t] for k in modelo.clientes)
    else:
        return (
            sum(modelo.PA[p, i, j, t] for i in modelo.productores)
            + modelo.InvCA[p, j, t - 1]
            == modelo.InvCA[p, j, t] + sum(modelo.AC[p, j, k, t] for k in modelo.clientes)
        )

modelo.flujo_acopio_constraint = pyo.Constraint(modelo.productos, modelo.centros_de_acopios, modelo.periodo_de_tiempo, rule=flujo_acopio_rule)

def binarias_operacion_rule(modelo, j, t):
    return sum(modelo.PA[p, i, j, t] for i in modelo.productores for p in modelo.productos) <= modelo.CA[j, t] * modelo.CapAlmacenamientoCA[j, p, t]

modelo.binarias_operacion_constraint = pyo.Constraint(modelo.centros_de_acopios, modelo.periodo_de_tiempo, rule=binarias_operacion_rule)

def asignacion_rule(modelo, i, p):
    return sum(modelo.AS[i, j, t] for j in modelo.centros_de_acopios for t in modelo.periodo_de_tiempo) == 1

modelo.asignacion_constraint = pyo.Constraint(modelo.productores, modelo.productos, rule=asignacion_rule)

def demanda_insatisfecha_rule(modelo, p, t):
    return (
        sum(modelo.AC[p, j, k, t] for j in modelo.centros_de_acopios for k in modelo.clientes) + modelo.DInsatisfecha[p, t]
        == sum(modelo.DemandaClie[p, k, t] for k in modelo.clientes)
    )

modelo.demanda_insatisfecha_constraint = pyo.Constraint(modelo.productos, modelo.periodo_de_tiempo, rule=demanda_insatisfecha_rule)



### 8. Función objetivo 

FUNCION OBJETIVO
\
\
$Min F$=  $CProducción_{t}$+$COperación_{t}$+$CInventario_{t}$+$CTransporte_{t}$+$DemandaInsatisfecha_{t}$ (1)




\begin{align*}
& CProduccion_t = \sum_{i \in I} \sum_{j \in J} \sum_{p \in P} \text{CostoProduccion}_{pit} \cdot \text{PA}_{pijt} \quad \forall t \in T \\
\\
& COperacion = \sum_{j \in J} \text{CostoFijoAcopio}_{jt} \cdot \text{CA}_{jt} + \sum_{i \in I} \sum_{j \in J} \sum_{p \in P} \text{CostoVarAcopio}_{pjt} \cdot \text{PA}_{pijt} \quad \forall t \in T \\
\\
& \text{Costos de inventario}_t = \sum_{j \in J} \sum_{p \in P} \text{CostoInvAcopio}_{pjt} \cdot \text{InvCA}_{pjt} \quad \forall t \in T \\
\\
& \text{Costo de transporte}_t = \sum_{j \in J} \sum_{i \in I} \sum_{p \in P} \text{CostoTransProdAcopio}_{pijt} \cdot \text{PA}_{pijt} + \sum_{k \in K} \sum_{j \in J} \sum_{p \in P} \text{CostoTransAcopioClie}_{pjkt} \cdot \text{AC}_{pjkt} \quad \forall t \in T \\
\\
& \text{DemandaInsatisfecha}_t = \sum_{p \in P} \text{Dinsatisfecha}_{pt} \cdot \text{PDinsatisfecha}_{pt} \quad \forall t \in T \\
\end{align*}




In [32]:
def objectivo_rule(modelo):
    return sum(modelo.CostoProduccion[p, i, t] * modelo.PA [p, i, j, t] for p in modelo.productos for i in modelo.productores for j in modelo.centros_de_acopios for t in modelo.periodo_de_tiempo) + \
           sum(modelo.CostoFijoAcopio[j, t] * modelo.CA[j, t] for j in modelo.centros_de_acopios for t in modelo.periodo_de_tiempo) + \
           sum(modelo.CostoVariableAcopio[p, j, t] * modelo.PA[p, i, j, t] for p in modelo.productos for i in modelo.productores for j in modelo.centros_de_acopios for t in modelo.periodo_de_tiempo) + \
           sum(modelo.CostoInventario[p, j, t] * modelo.InvCA[p, j, t] for p in modelo.productos for j in modelo.centros_de_acopios for t in modelo.periodo_de_tiempo) + \
           sum(modelo.CostoTransporteProducto[p, i, j, t] * modelo.PA[p, i, j, t] for p in modelo.productos for i in modelo.productores for j in modelo.centros_de_acopios for t in modelo.periodo_de_tiempo) + \
           sum(modelo.CostoTransAcopioCliente[p, j, k, t] * modelo.AC[p, j, k, t] for p in modelo.productos for j in modelo.centros_de_acopios for k in modelo.clientes for t in modelo.periodo_de_tiempo) + \
           sum(modelo.DInsatisfecha[p, t] * modelo.PDinsatisfecha[p, t] for p in modelo.productos for t in modelo.periodo_de_tiempo) 
modelo.objetivo = pyo.Objective(rule=objectivo_rule, sense=pyo.minimize)


### 9. Resuelve el modelo 

In [33]:
instancia_modelo = modelo.create_instance()
solucion = pyo.SolverFactory('glpk').solve(instancia_modelo)

ERROR: Rule failed for Param 'CostoProduccion' with index ('Queso duro',
'ROBERTO ANTONIO ANAYA', 'Enero'): TypeError: unsupported operand type(s) for
+: 'int' and 'type'
ERROR: Constructing component 'CostoProduccion' from data=None failed:
TypeError: unsupported operand type(s) for +: 'int' and 'type'


TypeError: unsupported operand type(s) for +: 'int' and 'type'

### 10. Imprime los resultados 

In [ ]:
print("Estado:", solucion.solver.status)
print("Costo total =", pyo.value(instancia_modelo.obj))

for v in instancia_modelo.component_objects(pyo.Var, active=True):
    print("Valores de", v)
    for index in v:
        print(f"{index}: {pyo.value(v[index])}")